#  Start notebook and load RoundManager

In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline
%matplotlib widget

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_columns = 300

from numebot.secret import PUBLIC_ID, SECRET_KEY
from numebot.env import NUMERAI_DATA_FOLDER, MODEL_CONFIGS_PATH

# Test numerapi, explore models' data and plot new monitoring figures.

In [ ]:
import numerapi
napi = numerapi.NumerAPI(verbosity="info", public_id=PUBLIC_ID, secret_key=SECRET_KEY)

In [ ]:
models_dict = napi.get_models()
models_dict

In [ ]:
napi.submission_status(models_dict['rpica']) is None
napi.get_current_round()
napi.check_round_open()

In [ ]:
leaderboard = napi.get_leaderboard(limit=20000)
len(leaderboard)
my_leaderboard = list(filter(lambda subm: 'rpica' in subm['username'], leaderboard))
my_leaderboard_df = pd.DataFrame(sorted(my_leaderboard, key=lambda x: x['corr20Rep'], reverse=True))

In [ ]:
my_leaderboard_df

In [ ]:
my_usernames = sorted(my_leaderboard_df['username'].unique())

performances = {}
for model_slot_name in my_usernames:
    response = napi.round_model_performances(model_slot_name)
    performances[model_slot_name] = pd.DataFrame(response)

In [ ]:
n=40
performances[model_slot_name].iloc[n:n+5]

In [ ]:
from numebot.monitoring_v2.metrics_plots import plot_time_metrics, plot_aggregated_metrics
plot_time_metrics(performances, my_usernames)

In [ ]:
plot_aggregated_metrics(performances, my_usernames, period_in_months=3)

# Run prediction and submission steps manually

In [ ]:
from numebot.data.data_constants import NC
from numebot.secret import PUBLIC_ID, SECRET_KEY
from numebot.env import NUMERAI_DATA_FOLDER, MODEL_CONFIGS_PATH
from numebot.round_manager import RoundManager


napi = numerapi.NumerAPI(verbosity="info", public_id=PUBLIC_ID, secret_key=SECRET_KEY)

rm = RoundManager(NUMERAI_DATA_FOLDER,
                  model_configs_path=MODEL_CONFIGS_PATH,
                  public_id=PUBLIC_ID, 
                  secret_key=SECRET_KEY, 
                  #nrows=10000, testing=True  # Uncomment to set testing mode.
                  )

# Get list of models with their model file
rm.models_info()

In [ ]:
# Get predictions for each model
rm.generate_predictions_for_all_models()

In [ ]:
# Submit predictions (test with rpica_test_3)
# Can I check if I submitted? (for example requesting the scoring)
rm.submit_predictions()

## Monitoring (use the dashboard)

In [ ]:
full_df = rm.mm.download_round_details()
#full_df = rm.mm.load_round_details_csv()
full_df.shape

In [ ]:
import numebot.monitoring.metrics_plotter as mp

_ = mp.plot_round_details(full_df, rounds=[262, 263])

_ = mp.plot_round_details(full_df, model_names=['rpica', 'rpica_test_1'])

## Check individual model

In [ ]:
rm.models_dict.keys()

In [ ]:
model = rm['rpica']

In [ ]:
status_df = model.get_weekly_submission_status(to_be_saved=False)
lb_df = model.get_daily_leaderboard(to_be_saved=False)

status_df.head(2)
lb_df.head(2)

In [ ]:
model.train_model(rm.data)

In [ ]:
model.predict(rm.data.tournament, to_be_saved_for_submission=False)

In [ ]:
model.submit_predictions()

## EDA

In [ ]:
for dataset, set_name in zip([rm.data.train, rm.data.val, rm.data.test, rm.data.live], ['train', 'val', 'test', 'live']):
    print(f'Info about {set_name}: shape {dataset.shape}')
    dataset.groupby('data_type')['era'].agg(
        ['count',
         'min',
         'max', 
         pd.Series.nunique, 
         lambda x: sorted(list(np.unique(x)))
        ])